In [1]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import Levenshtein
import difflib
import multiprocessing
import pandas as pd
from collections import Counter

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np
import lightgbm as lgb
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from fuzzywuzzy import fuzz
from fuzzywuzzy.fuzz import WRatio, partial_ratio, QRatio, token_set_ratio, token_sort_ratio, partial_token_sort_ratio

In [2]:
## Parameters
is_debug = False
SEED = 2022
num_neighbors = 25
num_split = 3

threshold = 2

feat_columns = ['name', 'address', 'city', 'state', 'zip', 'url', 'phone', 'categories', 'country']
vec_columns = ['name', 'categories', 'address', 'state', 'url', 'country']

recall_columns = ['name', 'address', 'categories', 'address', 'phone']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(SEED)

In [3]:
%load_ext Cython

In [4]:
%%cython
def LCS(str S, str T):
    cdef int i, j
    cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
    for i in range(len(S)):
        for j in range(len(T)):
            dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
    return dp[len(S)][len(T)]

In [5]:
def recall_simple(df, threshold):
    val2id_d = {}
    for col in recall_columns:
        temp_df = df[['id', col]]
        temp_df[col] = temp_df[col].str.lower()
        val2id = temp_df.groupby(col)['id'].apply(set).to_dict()
        val2id_d[col] = val2id
        del val2id
    
    cus_ids = []
    match_ids = []
    for vals in tqdm(df[recall_columns + ['id']].fillna('null').values):
        cus_id = vals[-1]
        match_id = []
        
        rec_match_count = []
        for i in range(len(recall_columns)):
            col = recall_columns[i]
            
            if vals[i] != 'null':
                rec_match_count += list(val2id_d[col][vals[i].lower()])
        rec_match_count = dict(Counter(rec_match_count))
        
        for k, v in rec_match_count.items():
            if v > threshold:
                match_id.append(k)
        
        cus_ids += [cus_id] * len(match_id)
        match_ids += match_id
    
    train_df = pd.DataFrame()
    train_df['id'] = cus_ids
    train_df['match_id'] = match_ids
    train_df = train_df.drop_duplicates()
    del cus_ids, match_ids
    
    num_data = len(train_df)
    num_data_per_id = num_data / train_df['id'].nunique()
    print('Num of data: %s' % num_data)
    print('Num of data per id: %s' % num_data_per_id)
    
    return train_df

In [6]:
def recall_knn(df, Neighbors = 10):
    print('Start knn grouped by country')
    train_df_country = []
    for country, country_df in tqdm(df.groupby('country')):
        country_df = country_df.reset_index(drop = True)

        neighbors = min(len(country_df), Neighbors)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(country_df[['latitude','longitude']], country_df.index)
        dists, nears = knn.kneighbors(country_df[['latitude', 'longitude']], 
                                        return_distance = True)

        for k in range(neighbors):            
            cur_df = country_df[['id']]
            cur_df['match_id'] = country_df['id'].values[nears[:, k]]
            cur_df['kdist_country'] = dists[:, k]
            cur_df['kneighbors_country'] = k
            
            train_df_country.append(cur_df)
    train_df_country = pd.concat(train_df_country)
    
    print('Start knn')
    train_df = []
    knn = NearestNeighbors(n_neighbors = Neighbors)
    knn.fit(df[['latitude','longitude']], df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']])
    
    for k in range(Neighbors):            
        cur_df = df[['id']]
        cur_df['match_id'] = df['id'].values[nears[:, k]]
        cur_df['kdist'] = dists[:, k]
        cur_df['kneighbors'] = k
        train_df.append(cur_df)
    
    train_df = pd.concat(train_df)
    train_df = train_df.merge(train_df_country,
                                 on = ['id', 'match_id'],
                                 how = 'outer')
    del train_df_country
    
    return train_df

In [7]:
def add_features(df):    
    for col in tqdm(['name', 'categories', 'address']):       
        
        col_values = data.loc[df['id']][col].values.astype(str)
        matcol_values = data.loc[df['match_id']][col].values.astype(str)
        w_ratio = []
        partialratio = []
        tokenset_ratio = []
        tokensort_ratio = []
        fuzz_power = []
        
        for s, match_s in zip(col_values, matcol_values):
            if s != 'nan' and match_s != 'nan':                    

                s = str(s).lower().split()
                match_s = str(match_s).lower().split()
                a, b = WRatio(s, match_s), partial_ratio(s, match_s)
                c, d = token_set_ratio(s, match_s), token_sort_ratio(s, match_s)
                w_ratio.append(a)
                partialratio.append(b)
                tokenset_ratio.append(c)
                tokensort_ratio.append(d)
                fuzz_power.append((a+b+c+d)/4.)
            else:
                w_ratio.append(np.nan)
                partialratio.append(np.nan)
                tokenset_ratio.append(np.nan)
                tokensort_ratio.append(np.nan)
                fuzz_power.append(np.nan)

        df[f'{col}_w_ratio'] = w_ratio
        df[f'{col}_partial_ratio'] = partialratio
        df[f'{col}_tokenset_ratio'] = tokenset_ratio
        df[f'{col}_tokensort_ratio'] = tokensort_ratio
        df[f'{col}_fuzz_power'] = fuzz_power
            
        gc.collect()
            
    return df

In [8]:
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def analysis(df):
    print('Num of data: %s' % len(df))
    print('Num of unique id: %s' % df['id'].nunique())
    print('Num of unique poi: %s' % df['point_of_interest'].nunique())
    
    poi_grouped = df.groupby('point_of_interest')['id'].count().reset_index()
    print('Mean num of unique poi: %s' % poi_grouped['id'].mean())

In [9]:
## Data load
data_root = '../src/data/raw/'  #'../input/foursquare-location-matching'
data = pd.read_csv(os.path.join(data_root, 'train.csv'))

if is_debug:
    data = data.sample(n = 10000, random_state = SEED)
    data = data.reset_index(drop = True)

In [10]:
## Data split
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(data, data['point_of_interest'], data['point_of_interest'])):
    data.loc[val_idx, 'set'] = i

print('Num of train data: %s' % len(data))
print(data['set'].value_counts())

Num of train data: 1138812
1.0    569406
0.0    569406
Name: set, dtype: int64


In [11]:
valid_data = data[data['set'] == 0]
train_data = data[data['set'] == 1]

print('Train data: ')
analysis(train_data)
print('Valid data: ')
analysis(valid_data)

train_poi = train_data['point_of_interest'].unique().tolist()
valid_poi = valid_data['point_of_interest'].unique().tolist()

print(set(train_poi) & set(valid_poi))

train_ids = train_data['id'].unique().tolist()
valid_ids = valid_data['id'].unique().tolist()
      
print(set(train_ids) & set(valid_ids))

Train data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369987
Mean num of unique poi: 1.5389892077289202
Valid data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369985
Mean num of unique poi: 1.5389975269267673
set()
set()


In [12]:
train_data.head().T

,0,10,11,12,13
id,E_000001272c6c5d,E_00009ab517afac,E_0000c362229d93,E_0000c566a81ea1,E_0000d9e584ed9f
name,Café Stad Oudenaarde,Starbucks,Coffee Cat,つじ田 味噌の章,Signature Properties Savannah
latitude,50.86,26.3052,7.08222,35.6948,32.0126
longitude,3.6342,50.1294,125.61,139.767,-81.1132
address,Abdijstraat,Ibis Avenue,F. Torres St.,神田小川町1-1,100 Commercial Ct Ste C
city,Nederename,Dhahran,Davao City,千代田区,Savannah
state,Oost-Vlaanderen,Ash Sharqiyah,Davao Region,東京都,GA
zip,9700,34465,8000,101-0052,31406
country,BE,SA,PH,JP,US
url,NaN,NaN,NaN,https://tsukemen-tsujita.com,http://www.oursignatureproperties.com


In [13]:
tv_ids_d = {}
tv_ids_d['train_ids'] = train_ids
tv_ids_d['valid_ids'] = valid_ids

np.save('tv_ids_d.npy', tv_ids_d)

del train_data, valid_data
gc.collect()

data = data.set_index('id')
data = data.loc[tv_ids_d['train_ids']]
data = data.reset_index()

In [14]:
data.shape

(569406, 14)

In [15]:
data.head().T

,0,1,2,3,4
id,E_000001272c6c5d,E_00009ab517afac,E_0000c362229d93,E_0000c566a81ea1,E_0000d9e584ed9f
name,Café Stad Oudenaarde,Starbucks,Coffee Cat,つじ田 味噌の章,Signature Properties Savannah
latitude,50.86,26.3052,7.08222,35.6948,32.0126
longitude,3.6342,50.1294,125.61,139.767,-81.1132
address,Abdijstraat,Ibis Avenue,F. Torres St.,神田小川町1-1,100 Commercial Ct Ste C
city,Nederename,Dhahran,Davao City,千代田区,Savannah
state,Oost-Vlaanderen,Ash Sharqiyah,Davao Region,東京都,GA
zip,9700,34465,8000,101-0052,31406
country,BE,SA,PH,JP,US
url,NaN,NaN,NaN,https://tsukemen-tsujita.com,http://www.oursignatureproperties.com


In [16]:
id2index_d = dict(zip(data['id'].values, data.index))

tfidf_d = {}
for col in vec_columns:
    tfidf = TfidfVectorizer()
    tv_fit = tfidf.fit_transform(data[col].fillna('nan'))
    tfidf_d[col] = tv_fit

In [17]:
tfidf_d['name']

<569406x235452 sparse matrix of type '<class 'numpy.float64'>'
	with 1519083 stored elements in Compressed Sparse Row format>

In [18]:
## Train data generated by Simple recall & knn recall
train_data_simple = recall_simple(data, threshold)
train_data = recall_knn(data, num_neighbors)

print('train data by knn: %s' % len(train_data))
train_data = train_data.merge(train_data_simple,
                             on = ['id', 'match_id'],
                             how = 'outer')
del train_data_simple
gc.collect()

  0%|          | 0/569406 [00:00<?, ?it/s]

Num of data: 1769808
Num of data per id: 4.739431527869274
Start knn grouped by country


  0%|          | 0/210 [00:00<?, ?it/s]

Start knn
train data by knn: 17517870


0

In [19]:
data = data.set_index('id')
ids = train_data['id'].tolist()
match_ids = train_data['match_id'].tolist()

poi = data.loc[ids]['point_of_interest'].values
match_poi = data.loc[match_ids]['point_of_interest'].values

train_data['label'] = np.array(poi == match_poi, dtype = np.int8)
del poi, match_poi, ids, match_ids
gc.collect()

print('Num of unique id: %s' % train_data['id'].nunique())
print('Num of train data: %s' % len(train_data))
print('Pos rate: %s' % train_data['label'].mean())

Num of unique id: 569406
Num of train data: 18840271
Pos rate: 0.054995758818968156


In [20]:
train_data.head().T

,0,1,2,3,4
id,E_000001272c6c5d,E_00009ab517afac,E_0000c362229d93,E_0000c566a81ea1,E_0000d9e584ed9f
match_id,E_000001272c6c5d,E_00009ab517afac,E_0000c362229d93,E_0000c566a81ea1,E_0000d9e584ed9f
kdist,0,0,0,0,0
kneighbors,0,0,0,0,0
kdist_country,0,0,0,0,0
kneighbors_country,0,0,0,0,0
label,1,1,1,1,1


In [21]:
train_data[train_data.label==0].head(5)

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label
261,E_0020ff8f3b06f9,E_241b3370406f67,0.0,0.0,NaN,NaN,0
351,E_002c767fde5d45,E_15d9faca67fdad,0.0,0.0,0.0,4.0,0
1571,E_00bbd9375a59eb,E_241b3370406f67,0.0,0.0,NaN,NaN,0
1680,E_00c720383fcdb5,E_cf5434e5494ebf,0.0,0.0,0.0,1.0,0
3635,E_01a3bbea9cd02f,E_caf447b497ce4e,0.0,0.0,0.0,1.0,0


In [22]:
train_data[train_data.label==1].head(5)

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label
0,E_000001272c6c5d,E_000001272c6c5d,0.0,0.0,0.0,0.0,1
1,E_00009ab517afac,E_00009ab517afac,0.0,0.0,0.0,0.0,1
2,E_0000c362229d93,E_0000c362229d93,0.0,0.0,0.0,0.0,1
3,E_0000c566a81ea1,E_0000c566a81ea1,0.0,0.0,0.0,0.0,1
4,E_0000d9e584ed9f,E_0000d9e584ed9f,0.0,0.0,0.0,0.0,1


In [23]:
train_data[(train_data.label==1) & (train_data.kneighbors_country!=0)].sort_values('kneighbors_country', 
                                                                                   ascending=False)

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label
14069913,E_b5718f72673423,E_e689b48b1bcd06,0.001637,24.0,0.001621,24.0,1
9352996,E_6cd54f1e53ddd1,E_75289d179b3690,0.058147,16.0,0.053489,24.0,1
14047585,E_ab6830163f36c0,E_b069b9b76de7e0,0.000084,24.0,0.000084,24.0,1
14047280,E_ab46c3cad7d5bf,E_ae4e33d9b84bbb,0.037658,24.0,0.033741,24.0,1
13384852,E_8195d0bf03fb22,E_a621faf329e014,0.000822,23.0,0.000712,24.0,1
...,...,...,...,...,...,...,...
18840263,E_fff9fde4bb588b,E_7fa50a9b5c716a,NaN,NaN,NaN,NaN,1
18840264,E_fff9fde4bb588b,E_110abe3244b781,NaN,NaN,NaN,NaN,1
18840265,E_fffa63a008cbcd,E_623477839dbabb,NaN,NaN,NaN,NaN,1
18840266,E_fffc94cbf08007,E_c6ae0edb8f697e,NaN,NaN,NaN,NaN,1


In [24]:
train_data[train_data.id=='E_1d029a2db783ca']

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label
64791,E_1d029a2db783ca,E_1d029a2db783ca,0.000000,0.0,0.000000,0.0,1
634197,E_1d029a2db783ca,E_a234ddf642795c,0.000599,1.0,0.000514,1.0,1
1203603,E_1d029a2db783ca,E_44917cedfbd5ec,0.000910,2.0,0.000838,3.0,1
1773009,E_1d029a2db783ca,E_57ade5df7f4692,0.001057,3.0,0.000817,2.0,1
2342415,E_1d029a2db783ca,E_e0e9af0727107a,0.003404,4.0,0.002632,4.0,0
2911821,E_1d029a2db783ca,E_fb3ec221a5f80c,0.003678,5.0,0.002939,5.0,0
3481227,E_1d029a2db783ca,E_98a0cd6756099e,0.005317,6.0,0.004119,6.0,0
4050633,E_1d029a2db783ca,E_8f68217136805f,0.005907,7.0,0.005546,7.0,0
4620039,E_1d029a2db783ca,E_24a99e772366b1,0.006802,8.0,0.005554,8.0,0
5189445,E_1d029a2db783ca,E_35fc9db907dbb7,0.007284,9.0,0.005871,9.0,0


In [25]:
## Eval
data = data.reset_index()

id2poi = get_id2poi(data)
poi2ids = get_poi2ids(data)

eval_df = pd.DataFrame()
eval_df['id'] = data['id'].unique().tolist()
eval_df['match_id'] = eval_df['id']
print('Unique id: %s' % len(eval_df))

eval_df_ = train_data[train_data['label'] == 1][['id', 'match_id']]
eval_df = pd.concat([eval_df, eval_df_])

eval_df = eval_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
eval_df['matches'] = eval_df['match_id'].apply(lambda x: ' '.join(set(x)))
print('Unique id: %s' % len(eval_df))

iou_score = get_score(eval_df)
print('IoU score: %s' % iou_score)

Unique id: 569406
Unique id: 569406
IoU score: 0.933545902937453


In [26]:
count = 0
start_row = 0

data = data.set_index('id')
unique_id = train_data['id'].unique().tolist()
num_split_id = len(unique_id) // num_split

In [27]:
num_split_id, len(unique_id), num_split

(189802, 569406, 3)

In [28]:
## Add features
for k in range(1, num_split + 1):
    print('Current split: %s' % k)
    end_row = start_row + num_split_id
    if k < num_split:
        cur_id = unique_id[start_row : end_row]
        cur_data = train_data[train_data['id'].isin(cur_id)]
    else:
        cur_id = unique_id[start_row: ]
        cur_data = train_data[train_data['id'].isin(cur_id)]
    
    cur_data = add_features(cur_data)
    print(cur_data.shape)
    print(cur_data.sample(1))
    
    cur_data.to_csv('../src/data/processed/train_fuzz_sr_25n_%s.csv' % k, index = False)    
    start_row = end_row
    count += len(cur_data)
    
    del cur_data
    gc.collect()
    
print(count)

Current split: 1


  0%|          | 0/3 [00:00<?, ?it/s]

(6278655, 22)
                       id          match_id     kdist  kneighbors  \
4081063  E_2ab708e8485a8b  E_5bf26475ab87e1  0.008698         7.0   

         kdist_country  kneighbors_country  label  name_w_ratio  \
4081063       0.007962                 7.0      0          42.0   

         name_partial_ratio  name_tokenset_ratio  name_tokensort_ratio  \
4081063                51.0                 36.0                  36.0   

         name_fuzz_power  categories_w_ratio  categories_partial_ratio  \
4081063            41.25                26.0                      55.0   

         categories_tokenset_ratio  categories_tokensort_ratio  \
4081063                       14.0                        14.0   

         categories_fuzz_power  address_w_ratio  address_partial_ratio  \
4081063                  27.25             49.0                   50.0   

         address_tokenset_ratio  address_tokensort_ratio  address_fuzz_power  
4081063                    29.0                     2

  0%|          | 0/3 [00:00<?, ?it/s]

(6276648, 22)
                        id          match_id  kdist  kneighbors  \
17163261  E_5cea75f28ef2a8  E_fe248055b18d34    NaN         NaN   

          kdist_country  kneighbors_country  label  name_w_ratio  \
17163261       0.010396                20.0      0          50.0   

          name_partial_ratio  name_tokenset_ratio  name_tokensort_ratio  \
17163261                54.0                 32.0                  32.0   

          name_fuzz_power  categories_w_ratio  categories_partial_ratio  \
17163261             42.0                50.0                      46.0   

          categories_tokenset_ratio  categories_tokensort_ratio  \
17163261                       25.0                        25.0   

          categories_fuzz_power  address_w_ratio  address_partial_ratio  \
17163261                   36.5             61.0                   64.0   

          address_tokenset_ratio  address_tokensort_ratio  address_fuzz_power  
17163261                    39.0              

  0%|          | 0/3 [00:00<?, ?it/s]

(6284968, 22)
                        id          match_id     kdist  kneighbors  \
12355588  E_b2b39ff67c9463  E_c9257942e7dfab  0.177104        21.0   

          kdist_country  kneighbors_country  label  name_w_ratio  \
12355588            NaN                 NaN      0          64.0   

          name_partial_ratio  name_tokenset_ratio  name_tokensort_ratio  \
12355588                67.0                 67.0                  53.0   

          name_fuzz_power  categories_w_ratio  categories_partial_ratio  \
12355588            62.75               100.0                     100.0   

          categories_tokenset_ratio  categories_tokensort_ratio  \
12355588                      100.0                       100.0   

          categories_fuzz_power  address_w_ratio  address_partial_ratio  \
12355588                  100.0              NaN                    NaN   

          address_tokenset_ratio  address_tokensort_ratio  address_fuzz_power  
12355588                     NaN        